In [1]:
import os
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

# load data

In [3]:
data_path = r"/home/alex/data/nlp/agmir/toy_data/trs_data"

In [4]:
import unicodedata
import string
import re
import random

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(os.path.join(data_path,'%s-%s.txt' % (lang1, lang2)), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
PAD_token = 0
SOS_token = 1
EOS_token = 2


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2:'EOS'}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [8]:
%%time
input_lang, output_lang, pairs = readLangs('eng', 'fra')

Reading lines...
CPU times: user 4.45 s, sys: 84 ms, total: 4.53 s
Wall time: 4.53 s


In [9]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [10]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [11]:
fra, eng, pairs = prepareData('eng', 'fra', True)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4346
eng 2804


In [22]:
def tokenize_quick_dirty(sent, lang):
    return [lang.word2index[p] for p in sent.split(' ')]

sent = 'j ai ans .'
tokenize_quick_dirty(sent, fra)

[3, 4, 5, 6]

In [23]:
processed_data_path = r"/home/alex/data/nlp/agmir/transf_processed_data"

with open(os.path.join(processed_data_path, 'fra', 'set.pkl'), 'wb') as f:
    pickle.dump([tokenize_quick_dirty(p[0], fra) for p in pairs], f)
    
with open(os.path.join(processed_data_path, 'fra', 'voc.pkl'), 'wb') as f:
    pickle.dump(fra.index2word, f)
    
with open(os.path.join(processed_data_path, 'eng', 'set.pkl'), 'wb') as f:
    pickle.dump([tokenize_quick_dirty(p[1], eng) for p in pairs], f)
    
with open(os.path.join(processed_data_path, 'eng', 'voc.pkl'), 'wb') as f:
    pickle.dump(eng.index2word, f)